<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/designite/analise_designite_activemq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepara ambiente

In [4]:
!java --version

openjdk 11.0.20 2023-07-18
OpenJDK Runtime Environment (build 11.0.20+8-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.20+8-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [5]:
!python3 --version

Python 3.10.12


In [6]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


## Funcões

In [7]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd activemq && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta nome_repositorio
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio nome_repositorio
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta nome_repositorio
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto nome_repositorio
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto nome_repositorio
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [8]:
def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['ACTIVEMQ'] = tag + 'activemq-broker/src/main/java'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $ACTIVEMQ -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [9]:
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

'''
Example usage:
url1 = "https://www.example.com"
url2 = "https://www.invalid-url.com"
print(is_valid_url(url1))  # Output: True
print(is_valid_url(url2))  # Output: False
'''

'\nExample usage:\nurl1 = "https://www.example.com"\nurl2 = "https://www.invalid-url.com"\nprint(is_valid_url(url1))  # Output: True\nprint(is_valid_url(url2))  # Output: False\n'

In [10]:
import re

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

# Example list of strings
strings_list = ['gson-parent-1.0', 'gson-parent-2.0', 'gson-child-1.0', 'gson-parent-3.0']
pattern_to_match = r'gson-parent'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

['gson-parent-1.0', 'gson-parent-2.0', 'gson-parent-3.0']


# Análise Designite

In [11]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [12]:
!git clone https://github.com/apache/activemq.git

Cloning into 'activemq'...
remote: Enumerating objects: 232708, done.
remote: Counting objects: 100% (1084/1084), done.
remote: Compressing objects: 100% (529/529), done.
remote: Total 232708 (delta 524), reused 948 (delta 457), pack-reused 231624
Receiving objects: 100% (232708/232708), 59.14 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (112165/112165), done.


In [10]:
!tree activemq

A saída de streaming foi truncada nas últimas 5000 linhas.
│       │   │               │   ├── SessionState.java
│       │   │               │   ├── Tracked.java
│       │   │               │   └── TransactionState.java
│       │   │               ├── thread
│       │   │               │   ├── DedicatedTaskRunner.java
│       │   │               │   ├── DefaultThreadPools.java
│       │   │               │   ├── DeterministicTaskRunner.java
│       │   │               │   ├── PooledTaskRunner.java
│       │   │               │   ├── Scheduler.java
│       │   │               │   ├── SchedulerTimerTask.java
│       │   │               │   ├── Task.java
│       │   │               │   ├── TaskRunnerFactory.java
│       │   │               │   └── TaskRunner.java
│       │   │               ├── ThreadPriorities.java
│       │   │               ├── transaction
│       │   │               │   └── Synchronization.java
│       │   │               ├── TransactionContext.java
│       │   │     

### Pega a lista de tags

In [43]:
lista_tags_activemq = !cd activemq && git tag
print(lista_tags_activemq)

['activemq-4.0', 'activemq-4.0-M4', 'activemq-4.0-RC1', 'activemq-4.0-RC2', 'activemq-4.0-RC3', 'activemq-4.0.1', 'activemq-4.0.2', 'activemq-4.1.0', 'activemq-4.1.1', 'activemq-4.1.2', 'activemq-5.0.0', 'activemq-5.1.0', 'activemq-5.10.0', 'activemq-5.10.1', 'activemq-5.10.2', 'activemq-5.11.0', 'activemq-5.11.1', 'activemq-5.11.2', 'activemq-5.11.3', 'activemq-5.11.4', 'activemq-5.12.0', 'activemq-5.12.1', 'activemq-5.12.2', 'activemq-5.12.3', 'activemq-5.13.0', 'activemq-5.13.1', 'activemq-5.13.2', 'activemq-5.13.3', 'activemq-5.13.4', 'activemq-5.13.5', 'activemq-5.14.0', 'activemq-5.14.1', 'activemq-5.14.2', 'activemq-5.14.3', 'activemq-5.14.4', 'activemq-5.14.5', 'activemq-5.15.0', 'activemq-5.15.1', 'activemq-5.15.10', 'activemq-5.15.11', 'activemq-5.15.12', 'activemq-5.15.13', 'activemq-5.15.14', 'activemq-5.15.15', 'activemq-5.15.2', 'activemq-5.15.3', 'activemq-5.15.4', 'activemq-5.15.5', 'activemq-5.15.6', 'activemq-5.15.7', 'activemq-5.15.8', 'activemq-5.15.9', 'activemq-5.

In [14]:
pattern_to_extract = r'activemq-5.15'
get_elements_with_pattern(lista_tags_activemq, pattern_to_extract)

['activemq-5.15.0',
 'activemq-5.15.1',
 'activemq-5.15.10',
 'activemq-5.15.11',
 'activemq-5.15.12',
 'activemq-5.15.13',
 'activemq-5.15.14',
 'activemq-5.15.15',
 'activemq-5.15.2',
 'activemq-5.15.3',
 'activemq-5.15.4',
 'activemq-5.15.5',
 'activemq-5.15.6',
 'activemq-5.15.7',
 'activemq-5.15.8',
 'activemq-5.15.9']

In [15]:
pattern_to_extract = r'activemq-5.18'
get_elements_with_pattern(lista_tags_activemq, pattern_to_extract)

['activemq-5.18.0', 'activemq-5.18.1', 'activemq-5.18.2']

# Análise da tag activemq_5_18_2

In [16]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_activemq_5_18_2 = get_commit_tag(tag='activemq-5.18.2')

In [17]:
cria_pasta_n_commit(repositorio='activemq', pasta_n_commit='activemq_5_18_2')

[]

In [18]:
!ls activemq_5_18_2

activemq-all		 activemq-log4j-appender      activemq-web
activemq-amqp		 activemq-mqtt		      activemq-web-console
activemq-blueprint	 activemq-openwire-generator  activemq-web-demo
activemq-broker		 activemq-openwire-legacy     assembly
activemq-cf		 activemq-osgi		      committer-tools
activemq-client		 activemq-pool		      doap.rdf
activemq-client-jakarta  activemq-ra		      docs
activemq-console	 activemq-rar		      etc
activemq-http		 activemq-run		      Jenkinsfile
activemq-jaas		 activemq-runtime-config      KEYS
activemq-jdbc-store	 activemq-shiro		      LICENSE
activemq-jms-pool	 activemq-spring	      NOTICE
activemq-kahadb-store	 activemq-stomp		      pom.xml
activemq-karaf		 activemq-tooling	      README.md
activemq-karaf-itest	 activemq-unit-tests	      SECURITY.md


In [19]:
checkout_n_commit('activemq_5_18_2', commit_tag_activemq_5_18_2[0])

Checkout do repositorio no commit 9cbf58d7bd420424004ae73659527747f9135676 realizado com sucesso!


In [20]:
ls -liath activemq_5_18_2/activemq-broker/src/main/java

total 12K
6298453 drwxr-xr-x 4 root root 4.0K Aug 24 18:21 ../
6298454 drwxr-xr-x 3 root root 4.0K Aug 24 18:21 ./
6298455 drwxr-xr-x 3 root root 4.0K Aug 24 18:21 org/


In [21]:
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-broker/src/main/java -o activemq_5_18_2

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "0b10000000"
Done.


In [22]:
!ls -liat activemq_5_18_2/designCodeSmells.csv

6298172 -rw-r--r-- 1 root root 49038 Aug 24 18:21 activemq_5_18_2/designCodeSmells.csv


In [45]:
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-jms-pool/src/main/java -o activemq_5_18_2/activemq-jms-pool
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-stomp/src/main/java -o activemq_5_18_2/activemq-stomp
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-web-console/src/main/java -o activemq_5_18_2/activemq-web-console
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-http/src/main/java -o activemq_5_18_2/activemq-http
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-ra/src/main/java -o activemq_5_18_2/activemq-ra
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-amqp/src/main/java -o activemq_5_18_2/activemq-amqp
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-cf/src/main/java -o activemq_5_18_2/activemq-cf
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-pool/src/main/java -o activemq_5_18_2/activemq-pool
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-karaf/src/main/java -o activemq_5_18_2/activemq-karaf
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-jaas/src/main/java -o activemq_5_18_2/activemq-jaas
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-log4j-appender/src/main/java -o activemq_5_18_2/activemq-log4j-appender
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-kahadb-store/src/main/java -o activemq_5_18_2/activemq-kahadb-store
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-web/src/main/java -o activemq_5_18_2/activemq-web
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-mqtt/src/main/java -o activemq_5_18_2/activemq-mqtt
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-runtime-config/src/main/java -o activemq_5_18_2/activemq-runtime-config
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-openwire-legacy/src/main/java -o activemq_5_18_2/activemq-openwire-legacy
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-broker/src/main/java -o activemq_5_18_2/activemq-broker
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-client/src/main/java -o activemq_5_18_2/activemq-client
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-openwire-generator/src/main/java -o activemq_5_18_2/activemq-openwire-generator
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-osgi/src/main/java -o activemq_5_18_2/activemq-osgi
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-shiro/src/main/java -o activemq_5_18_2/activemq-shiro
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-spring/src/main/java -o activemq_5_18_2/activemq-spring
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-tooling/activemq-maven-plugin/src/main/java -o activemq_5_18_2/activemq-tooling/activemq-maven-plugin
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-tooling/activemq-perf-maven-plugin/src/main/java -o activemq_5_18_2/activemq-tooling/activemq-perf-maven-plugin
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-tooling/activemq-joram-jms-tests/src/main/java -o activemq_5_18_2/activemq-tooling/activemq-joram-jms-tests
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-tooling/activemq-junit/src/main/java -o activemq_5_18_2/activemq-tooling/activemq-junit
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-tooling/activemq-memtest-maven-plugin/src/main/java -o activemq_5_18_2/activemq-tooling/activemq-memtest-maven-plugin
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-console/src/main/java -o activemq_5_18_2/activemq-console
!java -jar DesigniteJava.jar -i activemq_5_18_2/activemq-jdbc-store/src/main/java -o activemq_5_18_2/activemq-jdbc-store

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Ex

In [23]:
df_design_smells = pd.read_csv('activemq_5_18_2/designCodeSmells.csv')

In [48]:
df_design_smells1 = pd.read_csv('activemq_5_18_2/activemq-jms-pool/designCodeSmells.csv')
df_design_smells2 = pd.read_csv('activemq_5_18_2/activemq-stomp/designCodeSmells.csv')
df_design_smells3 = pd.read_csv('activemq_5_18_2/activemq-web-console/designCodeSmells.csv')
df_design_smells4 = pd.read_csv('activemq_5_18_2/activemq-http/designCodeSmells.csv')
df_design_smells5 = pd.read_csv('activemq_5_18_2/activemq-ra/designCodeSmells.csv')
df_design_smells6 = pd.read_csv('activemq_5_18_2/activemq-amqp/designCodeSmells.csv')
df_design_smells7 = pd.read_csv('activemq_5_18_2/activemq-cf/designCodeSmells.csv')
df_design_smells8 = pd.read_csv('activemq_5_18_2/activemq-pool/designCodeSmells.csv')
df_design_smells9 = pd.read_csv('activemq_5_18_2/activemq-karaf/designCodeSmells.csv')
df_design_smells10 = pd.read_csv('activemq_5_18_2/activemq-jaas/designCodeSmells.csv')
df_design_smells11 = pd.read_csv('activemq_5_18_2/activemq-log4j-appender/designCodeSmells.csv')
df_design_smells12 = pd.read_csv('activemq_5_18_2/activemq-kahadb-store/designCodeSmells.csv')
df_design_smells13 = pd.read_csv('activemq_5_18_2/activemq-web/designCodeSmells.csv')
df_design_smells14 = pd.read_csv('activemq_5_18_2/activemq-mqtt/designCodeSmells.csv')
df_design_smells15 = pd.read_csv('activemq_5_18_2/activemq-runtime-config/designCodeSmells.csv')
df_design_smells16 = pd.read_csv('activemq_5_18_2/activemq-openwire-legacy/designCodeSmells.csv')
df_design_smells17 = pd.read_csv('activemq_5_18_2/activemq-broker/designCodeSmells.csv')
df_design_smells18 = pd.read_csv('activemq_5_18_2/activemq-client/designCodeSmells.csv')
df_design_smells19 = pd.read_csv('activemq_5_18_2/activemq-openwire-generator/designCodeSmells.csv')
df_design_smells20 = pd.read_csv('activemq_5_18_2/activemq-osgi/designCodeSmells.csv')
df_design_smells21 = pd.read_csv('activemq_5_18_2/activemq-shiro/designCodeSmells.csv')
df_design_smells22 = pd.read_csv('activemq_5_18_2/activemq-spring/designCodeSmells.csv')
df_design_smells23 = pd.read_csv('activemq_5_18_2/activemq-tooling/activemq-maven-plugin/designCodeSmells.csv')
df_design_smells24 = pd.read_csv('activemq_5_18_2/activemq-tooling/activemq-perf-maven-plugin/designCodeSmells.csv')
#df_design_smells25 = pd.read_csv('activemq_5_18_2/activemq-tooling/activemq-joram-jms-tests/designCodeSmells.csv')
df_design_smells26 = pd.read_csv('activemq_5_18_2/activemq-tooling/activemq-junit/designCodeSmells.csv')
df_design_smells27 = pd.read_csv('activemq_5_18_2/activemq-tooling/activemq-memtest-maven-plugin/designCodeSmells.csv')
df_design_smells28 = pd.read_csv('activemq_5_18_2/activemq-console/designCodeSmells.csv')
df_design_smells29 = pd.read_csv('activemq_5_18_2/activemq-jdbc-store/designCodeSmells.csv')

In [53]:
dataframes = [df_design_smells1, df_design_smells2, df_design_smells3, df_design_smells4, df_design_smells5, df_design_smells6, df_design_smells7, df_design_smells8, df_design_smells9, df_design_smells10, df_design_smells11, df_design_smells12, df_design_smells13, df_design_smells14, df_design_smells15, df_design_smells16, df_design_smells17, df_design_smells18, df_design_smells19, df_design_smells20, df_design_smells21, df_design_smells22, df_design_smells23, df_design_smells24, df_design_smells26, df_design_smells27, df_design_smells28, df_design_smells29]

In [54]:
# Merge the dataframes
merged_dataframe = pd.concat(dataframes, axis=0, ignore_index=True)

In [64]:
merged_dataframe

,Project Name,Package Name,Type Name,Code Smell
0,java,org.apache.activemq.jms.pool,XaPooledConnectionFactory,Unutilized Abstraction
1,java,org.apache.activemq.jms.pool,PooledConnectionFactory,Unutilized Abstraction
2,java,org.apache.activemq.jms.pool,PooledConnectionFactory,Insufficient Modularization
3,java,org.apache.activemq.jms.pool,XaConnectionPool,Unutilized Abstraction
4,java,org.apache.activemq.jms.pool,Synchronization,Unutilized Abstraction
...,...,...,...,...
2839,java,org.apache.activemq.store.journal,TxOperation,Cyclic-Dependent Modularization
2840,java,org.apache.activemq.store.journal,Tx,Cyclic-Dependent Modularization
2841,java,org.apache.activemq.store.journal,JournalPersistenceAdapterFactory,Unutilized Abstraction
2842,java,org.apache.activemq.store.journal,JournalPersistenceAdapterFactory,Broken Hierarchy


In [65]:
df_design_smells = merged_dataframe.copy()
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,java,org.apache.activemq.jms.pool,XaPooledConnectionFactory,Unutilized Abstraction
1,java,org.apache.activemq.jms.pool,PooledConnectionFactory,Unutilized Abstraction
2,java,org.apache.activemq.jms.pool,PooledConnectionFactory,Insufficient Modularization
3,java,org.apache.activemq.jms.pool,XaConnectionPool,Unutilized Abstraction
4,java,org.apache.activemq.jms.pool,Synchronization,Unutilized Abstraction
...,...,...,...,...
2839,java,org.apache.activemq.store.journal,TxOperation,Cyclic-Dependent Modularization
2840,java,org.apache.activemq.store.journal,Tx,Cyclic-Dependent Modularization
2841,java,org.apache.activemq.store.journal,JournalPersistenceAdapterFactory,Unutilized Abstraction
2842,java,org.apache.activemq.store.journal,JournalPersistenceAdapterFactory,Broken Hierarchy


In [66]:
df_design_smells['Type Name'].unique()

array(['XaPooledConnectionFactory', 'PooledConnectionFactory',
       'XaConnectionPool', ..., 'JournalTransactionStore', 'TxOperation',
       'JournalPersistenceAdapterFactory'], dtype=object)

In [58]:
df_activemq_criticos = pd.read_csv('https://raw.githubusercontent.com/armandossrecife/piloto/main/criticos/activemq/activemq_arquivos_criticos.csv')
df_activemq_criticos.nome.to_list()

['BrokerService.java',
 'DemandForwardingBridgeSupport.java',
 'MessageDatabase.java',
 'Queue.java',
 'TopicSubscription.java',
 'TransportConnector.java']

In [67]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
lista_classes_criticas = df_activemq_criticos.nome.to_list()
lista_classes_criticas = ['BrokerService',
 'DemandForwardingBridgeSupport',
 'MessageDatabase',
 'Queue',
 'TopicSubscription',
 'TransportConnector']

In [68]:
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [69]:
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell
5,java,org.apache.activemq.jms.pool,PooledQueueSender,Unutilized Abstraction
31,java,org.apache.activemq.transport.stomp,StompQueueBrowserSubscription,Unutilized Abstraction
98,java,org.apache.activemq.transport.http,BlockingQueueTransport,Deficient Encapsulation
257,java,org.apache.activemq.store.kahadb,MessageDatabase,Deficient Encapsulation
258,java,org.apache.activemq.store.kahadb,MessageDatabase,Insufficient Modularization
...,...,...,...,...
2682,java,org.apache.activemq.junit,ActiveMQDynamicQueueSenderResource,Unutilized Abstraction
2688,java,org.apache.activemq.junit,ActiveMQQueueReceiverResource,Unutilized Abstraction
2689,java,org.apache.activemq.junit,ActiveMQQueueReceiverResource,Broken Hierarchy
2690,java,org.apache.activemq.junit,ActiveMQQueueSenderResource,Unutilized Abstraction


In [70]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
1116,java,org.apache.activemq.memory.buffer,MessageQueue,Cyclic-Dependent Modularization
1130,java,org.apache.activemq.broker,BrokerService,Cyclic-Dependent Modularization
1132,java,org.apache.activemq.broker,BrokerService,Hub-like Modularization
1137,java,org.apache.activemq.broker,BrokerServiceAware,Cyclic-Dependent Modularization
1175,java,org.apache.activemq.broker,TransportConnector,Cyclic-Dependent Modularization
1196,java,org.apache.activemq.broker.region,Queue,Cyclic-Dependent Modularization
1210,java,org.apache.activemq.broker.region,QueueDispatchSelector,Cyclic-Dependent Modularization
1217,java,org.apache.activemq.broker.region,TopicSubscription,Cyclic-Dependent Modularization
1223,java,org.apache.activemq.broker.region,QueueBrowserSubscription,Cyclic-Dependent Modularization
1225,java,org.apache.activemq.broker.region,QueueSubscription,Cyclic-Dependent Modularization


In [71]:
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('activemq_5_18_2/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('activemq_5_18_2/analise_designite.csv', index=False)

In [82]:
!cd activemq_5_18_2 && zip ds_activemq_5_18_2.zip *.csv

updating: analise_designite_classes_criticas.csv (deflated 78%)
updating: analise_designite.csv (deflated 93%)
updating: designCodeSmells.csv (deflated 91%)
updating: implementationCodeSmells.csv (deflated 92%)
updating: methodMetrics.csv (deflated 91%)
updating: typeMetrics.csv (deflated 80%)


# Análise da tag activemq-5.15.0


In [33]:
commit_tag_activemq_5_15_0 = get_commit_tag(tag='activemq-5.15.0')

In [34]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
cria_pasta_n_commit(repositorio='activemq', pasta_n_commit='activemq_5_15_0')

[]

In [35]:
checkout_n_commit('activemq_5_15_0', commit_tag_activemq_5_15_0[0])

Checkout do repositorio no commit 5f0d6943cb97b0570bf8d58e8eaf9f0003a5cb6c realizado com sucesso!


In [36]:
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-broker/src/main/ -o activemq_5_15_0

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Exception while parsing literal number (during Magic Number detection). For input string: "0b10000000"
Done.


In [49]:
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-jms-pool/src/main/java -o activemq_5_15_0/activemq-jms-pool
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-stomp/src/main/java -o activemq_5_15_0/activemq-stomp
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-web-console/src/main/java -o activemq_5_15_0/activemq-web-console
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-http/src/main/java -o activemq_5_15_0/activemq-http
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-ra/src/main/java -o activemq_5_15_0/activemq-ra
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-amqp/src/main/java -o activemq_5_15_0/activemq-amqp
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-cf/src/main/java -o activemq_5_15_0/activemq-cf
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-pool/src/main/java -o activemq_5_15_0/activemq-pool
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-karaf/src/main/java -o activemq_5_15_0/activemq-karaf
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-jaas/src/main/java -o activemq_5_15_0/activemq-jaas
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-log4j-appender/src/main/java -o activemq_5_15_0/activemq-log4j-appender
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-kahadb-store/src/main/java -o activemq_5_15_0/activemq-kahadb-store
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-web/src/main/java -o activemq_5_15_0/activemq-web
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-mqtt/src/main/java -o activemq_5_15_0/activemq-mqtt
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-runtime-config/src/main/java -o activemq_5_15_0/activemq-runtime-config
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-openwire-legacy/src/main/java -o activemq_5_15_0/activemq-openwire-legacy
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-broker/src/main/java -o activemq_5_15_0/activemq-broker
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-client/src/main/java -o activemq_5_15_0/activemq-client
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-openwire-generator/src/main/java -o activemq_5_15_0/activemq-openwire-generator
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-osgi/src/main/java -o activemq_5_15_0/activemq-osgi
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-shiro/src/main/java -o activemq_5_15_0/activemq-shiro
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-spring/src/main/java -o activemq_5_15_0/activemq-spring
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-tooling/activemq-maven-plugin/src/main/java -o activemq_5_15_0/activemq-tooling/activemq-maven-plugin
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-tooling/activemq-perf-maven-plugin/src/main/java -o activemq_5_15_0/activemq-tooling/activemq-perf-maven-plugin
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-tooling/activemq-joram-jms-tests/src/main/java -o activemq_5_15_0/activemq-tooling/activemq-joram-jms-tests
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-tooling/activemq-junit/src/main/java -o activemq_5_15_0/activemq-tooling/activemq-junit
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-tooling/activemq-memtest-maven-plugin/src/main/java -o activemq_5_15_0/activemq-tooling/activemq-memtest-maven-plugin
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-console/src/main/java -o activemq_5_15_0/activemq-console
!java -jar DesigniteJava.jar -i activemq_5_15_0/activemq-jdbc-store/src/main/java -o activemq_5_15_0/activemq-jdbc-store

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.
Parsing the source code ...
Resolving symbols...
Extracting metrics...
Ex

In [37]:
df_design_smells_final = pd.read_csv('activemq_5_15_0/designCodeSmells.csv')

In [50]:
df_design_i_smells1 = pd.read_csv('activemq_5_15_0/activemq-jms-pool/designCodeSmells.csv')
df_design_i_smells2 = pd.read_csv('activemq_5_15_0/activemq-stomp/designCodeSmells.csv')
df_design_i_smells3 = pd.read_csv('activemq_5_15_0/activemq-web-console/designCodeSmells.csv')
df_design_i_smells4 = pd.read_csv('activemq_5_15_0/activemq-http/designCodeSmells.csv')
df_design_i_smells5 = pd.read_csv('activemq_5_15_0/activemq-ra/designCodeSmells.csv')
df_design_i_smells6 = pd.read_csv('activemq_5_15_0/activemq-amqp/designCodeSmells.csv')
df_design_i_smells7 = pd.read_csv('activemq_5_15_0/activemq-cf/designCodeSmells.csv')
df_design_i_smells8 = pd.read_csv('activemq_5_15_0/activemq-pool/designCodeSmells.csv')
df_design_i_smells9 = pd.read_csv('activemq_5_15_0/activemq-karaf/designCodeSmells.csv')
df_design_i_smells10 = pd.read_csv('activemq_5_15_0/activemq-jaas/designCodeSmells.csv')
df_design_i_smells11 = pd.read_csv('activemq_5_15_0/activemq-log4j-appender/designCodeSmells.csv')
df_design_i_smells12 = pd.read_csv('activemq_5_15_0/activemq-kahadb-store/designCodeSmells.csv')
df_design_i_smells13 = pd.read_csv('activemq_5_15_0/activemq-web/designCodeSmells.csv')
df_design_i_smells14 = pd.read_csv('activemq_5_15_0/activemq-mqtt/designCodeSmells.csv')
df_design_i_smells15 = pd.read_csv('activemq_5_15_0/activemq-runtime-config/designCodeSmells.csv')
df_design_i_smells16 = pd.read_csv('activemq_5_15_0/activemq-openwire-legacy/designCodeSmells.csv')
df_design_i_smells17 = pd.read_csv('activemq_5_15_0/activemq-broker/designCodeSmells.csv')
df_design_i_smells18 = pd.read_csv('activemq_5_15_0/activemq-client/designCodeSmells.csv')
df_design_i_smells19 = pd.read_csv('activemq_5_15_0/activemq-openwire-generator/designCodeSmells.csv')
df_design_i_smells20 = pd.read_csv('activemq_5_15_0/activemq-osgi/designCodeSmells.csv')
df_design_i_smells21 = pd.read_csv('activemq_5_15_0/activemq-shiro/designCodeSmells.csv')
df_design_i_smells22 = pd.read_csv('activemq_5_15_0/activemq-spring/designCodeSmells.csv')
df_design_i_smells23 = pd.read_csv('activemq_5_15_0/activemq-tooling/activemq-maven-plugin/designCodeSmells.csv')
df_design_i_smells24 = pd.read_csv('activemq_5_15_0/activemq-tooling/activemq-perf-maven-plugin/designCodeSmells.csv')
#df_design_i_smells25 = pd.read_csv('activemq_5_15_0/activemq-tooling/activemq-joram-jms-tests/designCodeSmells.csv')
df_design_i_smells26 = pd.read_csv('activemq_5_15_0/activemq-tooling/activemq-junit/designCodeSmells.csv')
df_design_i_smells27 = pd.read_csv('activemq_5_15_0/activemq-tooling/activemq-memtest-maven-plugin/designCodeSmells.csv')
df_design_i_smells28 = pd.read_csv('activemq_5_15_0/activemq-console/designCodeSmells.csv')
df_design_i_smells29 = pd.read_csv('activemq_5_15_0/activemq-jdbc-store/designCodeSmells.csv')

In [73]:
dataframes_i = [df_design_i_smells1, df_design_i_smells2, df_design_i_smells3, df_design_i_smells4, df_design_i_smells5, df_design_i_smells6, df_design_i_smells7, df_design_i_smells8, df_design_i_smells9, df_design_i_smells10, df_design_i_smells11, df_design_i_smells12, df_design_i_smells13, df_design_i_smells14, df_design_i_smells15, df_design_i_smells16, df_design_i_smells17, df_design_i_smells18, df_design_i_smells19, df_design_i_smells20, df_design_i_smells21, df_design_i_smells22, df_design_i_smells23, df_design_i_smells24, df_design_i_smells26, df_design_i_smells27, df_design_i_smells28, df_design_i_smells29]


In [74]:
# Merge the dataframes
merged_dataframe_i = pd.concat(dataframes_i, axis=0, ignore_index=True)

In [75]:
df_design_smells_final_i = merged_dataframe_i.copy()
df_design_smells_final_i

,Project Name,Package Name,Type Name,Code Smell
0,java,org.apache.activemq.jms.pool,XaPooledConnectionFactory,Unutilized Abstraction
1,java,org.apache.activemq.jms.pool,PooledConnectionFactory,Unutilized Abstraction
2,java,org.apache.activemq.jms.pool,PooledConnectionFactory,Insufficient Modularization
3,java,org.apache.activemq.jms.pool,XaConnectionPool,Unutilized Abstraction
4,java,org.apache.activemq.jms.pool,Synchronization,Unutilized Abstraction
...,...,...,...,...
2795,java,org.apache.activemq.store.journal,TxOperation,Cyclic-Dependent Modularization
2796,java,org.apache.activemq.store.journal,Tx,Cyclic-Dependent Modularization
2797,java,org.apache.activemq.store.journal,JournalPersistenceAdapterFactory,Unutilized Abstraction
2798,java,org.apache.activemq.store.journal,JournalPersistenceAdapterFactory,Broken Hierarchy


In [77]:
pesquisa_final = df_design_smells_final_i['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [78]:
df_critical_classes_final_i = df_design_smells_final_i[pesquisa_final]
df_critical_classes_final_i.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final_i = df_critical_classes_final_i.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [79]:
df_critical_classes_final_i

,ProjectName,PackageName,TypeName,CodeSmell
1100,java,org.apache.activemq.memory.buffer,MessageQueue,Cyclic-Dependent Modularization
1114,java,org.apache.activemq.broker,BrokerService,Cyclic-Dependent Modularization
1116,java,org.apache.activemq.broker,BrokerService,Hub-like Modularization
1121,java,org.apache.activemq.broker,BrokerServiceAware,Cyclic-Dependent Modularization
1159,java,org.apache.activemq.broker,TransportConnector,Cyclic-Dependent Modularization
1178,java,org.apache.activemq.broker.region,Queue,Cyclic-Dependent Modularization
1193,java,org.apache.activemq.broker.region,QueueDispatchSelector,Cyclic-Dependent Modularization
1200,java,org.apache.activemq.broker.region,TopicSubscription,Cyclic-Dependent Modularization
1205,java,org.apache.activemq.broker.region,QueueBrowserSubscription,Cyclic-Dependent Modularization
1207,java,org.apache.activemq.broker.region,QueueSubscription,Cyclic-Dependent Modularization


In [80]:
df_analise_designite_critical_classes_final_i = df_critical_classes_final_i.sort_values(by='TypeName')
df_analise_designite_critical_classes_final_i.to_csv('activemq_5_15_0/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final_i.to_csv('activemq_5_15_0/analise_designite.csv', index=False)

In [81]:
!cd activemq_5_15_0 && zip ds_activemq_5_15_0.zip *.csv

  adding: analise_designite_classes_criticas.csv (deflated 78%)
  adding: analise_designite.csv (deflated 93%)
  adding: designCodeSmells.csv (deflated 91%)
  adding: implementationCodeSmells.csv (deflated 92%)
  adding: methodMetrics.csv (deflated 91%)
  adding: typeMetrics.csv (deflated 80%)
